In [1]:
from datetime import datetime, timezone
import webbrowser

import requests
from dateutil.parser import parse as parse_date
from funcy import post_processing, project
from ipyleaflet import Map, Marker, LayerGroup, Popup
from ipywidgets import HTML

In [2]:
movie = 'the-current-war'
lat, lng = 51.525609042041864, -0.12358503246188147
params = {
    'include[0]': 'theaters',
    'include[1]': 'exhibitors',
    'include[2]': 'movies',
    'include[3]': 'formats',
    'filter[titles][slug][0]': movie,
    'filter[theaters][lat]': lat,
    'filter[theaters][lon]': lng,
    'filter[theaters][rf]': '3',
    'filter[theaters][country][0]': 'gu',
    'filter[theaters][country][1]': 'gg',
    'filter[theaters][country][2]': 'im',
    'filter[theaters][country][3]': 'gb',
    'filter[theaters][country][4]': 'je',
    'filter[theaters][country][5]': 'ie',
}

base_url = f'https://stdata.powster.com/screenings'

In [3]:
response = requests.get(base_url, params=params)
response.raise_for_status()
doc = response.json()

In [4]:
for key, value in doc.items():
    print(key, len(value))

data 3103
meta 0
included 240


In [5]:
@post_processing(list)
def get_screenings(data):
    seen = set()
    for screening in data:
        s = screening['attributes']
        if not s['date'] or not s['time']:
            continue
        time = parse_date(s['time'])
        date = parse_date(s['date'])
        dt = datetime.combine(date, time.time())
        s['datetime'] = dt.replace(tzinfo=dt.tzinfo or timezone.utc)
        unique_data = project(s, ('datetime', 'theater')).__str__()
        if unique_data in seen:
            continue
        seen.add(unique_data)
        yield s
    
screenings = get_screenings(doc['data'])
project(screenings[0], ('date', 'time', 'datetime'))

{'date': '2019-07-26T00:00:00.000Z',
 'time': '15:30:00',
 'datetime': datetime.datetime(2019, 7, 26, 15, 30, tzinfo=datetime.timezone.utc)}

In [6]:
@post_processing(list)
def filter_by_date(screenings, start, end):
    for s in screenings:
        if start <= s['datetime'] <= end:
            yield s

start = datetime(2019, 7, 29, 17, tzinfo=timezone.utc)
end = datetime(2019, 7, 29, 19, tzinfo=timezone.utc)
dated = filter_by_date(screenings, start, end)
dated[0]

{'movie_id': 'd097ac02-7d5d-11e9-8537-a7e0e3d26e93',
 'theater_id': '96b30dee-347c-11e9-8e8e-9732a0d60e15',
 'provider_id': 'wwm',
 'provider_screening_id': None,
 'date': '2019-07-29T00:00:00.000Z',
 'time': '18:15:00',
 'url': 'https://www.myvue.com/book-tickets/summary/10096/226797/2939?SC_CMP=AFF_POW',
 'cta': None,
 'is_visible': True,
 'tz_offset': None,
 'doc': None,
 'datetime': datetime.datetime(2019, 7, 29, 18, 15, tzinfo=datetime.timezone.utc)}

In [7]:
doc['included'][0]

{'type': 'movies',
 'id': 'd097ac02-7d5d-11e9-8537-a7e0e3d26e93',
 'attributes': {'provider_id': 'wwm',
  'provider_movie_id': '247746',
  'format_id': '4c5e546c-2d46-11e8-9156-9f1ab0ac9b09',
  'title': 'The Current War',
  'official_site': None,
  'fandango_movie_id': None,
  'movietickets_movie_id': None,
  'movie_master_id': 'bc9fedd4-0d59-11e7-8cd1-97388f63a017',
  'release_date': '2019-10-11T00:00:00.000Z',
  'meta': {'cast': ['Benedict Cumberbatch',
    'Michael Shannon',
    'Nicholas Hoult',
    'Katherine Waterston',
    'Tom Holland',
    'Tuppence Middleton'],
   'genre': ['Drama'],
   'rating': {'us': 'PG-13'},
   'writer': ['Michael Mitnick'],
   'director': ['Alfonso Gomez-Rejon'],
   'parent_id': '0',
   'distributor': ['101 Studios'],
   'webedia_trailer_file': '247746',
   'webedia_onesheet_filename': '247746h1.jpg'},
  'slug': 'the-current-war'}}

In [8]:
theaters = {item['id']: item['attributes'] for item in doc['included'] if item['type'] == 'theaters'}
next(iter(theaters.values()))

{'theater_master_id': '3b521d7c-f78f-11e6-b016-4f231659d920',
 'name': 'Cineworld Cinema - Harlow Town Centre',
 'address': 'Harvey Shopping Centre',
 'city': 'Harlow',
 'postcode': 'CM20 1XR',
 'country': 'gb',
 'state': 'UK',
 'url': 'https://www.cineworld.co.uk/cinemas/harlow-harvey-centre',
 'name_en': None,
 'address_en': None,
 'city_en': None,
 'lat': 51.77328,
 'lon': 0.08922,
 'tz': 'Europe/London',
 'tmsid': None,
 'tms_id': None,
 'fandango_theater_id': None,
 'movietickets_theater_id': None,
 'atom_theater_id': None,
 'wwm_ticketing': False,
 'wwm_tickettype': None,
 'exhibitor_id': 'e7f584a2-cfe9-11e5-b59b-73bc65f6099b',
 'provider_id': 'wwm',
 'provider_theater_id': '43108',
 'provider_exhibitor_id': None}

In [28]:
@post_processing(list)
def include_theaters(screenings):
    for s in screenings:
        theater_id = s.get('theater_id')
        if not theater_id:
            continue
        theater = theaters[theater_id]
        s['theater'] = theater
        s['map_title'] = f'{s["datetime"].strftime("%H:%M")} {s["theater"]["name"]}'
        yield s

theater_screenings = include_theaters(dated)
theater_screenings[0]['datetime'], theater_screenings[0]['theater']

(datetime.datetime(2019, 7, 29, 18, 15, tzinfo=datetime.timezone.utc),
 {'theater_master_id': 'e6efc518-62a8-11e9-9035-47f88dc7fa65',
  'name': 'Vue - Eltham',
  'address': '168-176 High Street',
  'city': 'Eltham',
  'postcode': 'SE9 1BJ',
  'country': 'gb',
  'state': 'UK',
  'url': 'https://www.myvue.com/',
  'name_en': None,
  'address_en': None,
  'city_en': None,
  'lat': 51.45068,
  'lon': 0.05645,
  'tz': None,
  'tmsid': None,
  'tms_id': None,
  'fandango_theater_id': None,
  'movietickets_theater_id': None,
  'atom_theater_id': None,
  'wwm_ticketing': False,
  'wwm_tickettype': None,
  'exhibitor_id': 'cdc4db50-cfe9-11e5-bf2d-c7ec2f75ebc8',
  'provider_id': 'wwm',
  'provider_theater_id': '49056',
  'provider_exhibitor_id': None})

In [29]:
time_to_title = [s['map_title'] for s in theater_screenings]
for s in sorted(time_to_title):
    print(s)

17:00 Showcase Newham
17:05 Vue - Westfield Stratford City
17:10 Vue - Westfield
17:15 Premiere Cinemas - Romford
17:20 Empire - Sutton
17:30 Cineworld Cinema - South Ruislip
17:40 Empire - Walthamstow
17:45 ODEON Swiss Cottage
17:50 ODEON Wimbledon
17:55 Vue - Harrow
18:00 Vue - Wood Green
18:05 Vue - Fulham Broadway
18:10 Vue - Thurrock
18:15 Vue - Eltham
18:20 Cineworld Cinema - Ilford
18:25 Vue - Shepherds Bush
18:30 Everyman Esher
18:45 Picturehouse Central
18:50 Vue - Watford


In [34]:
@post_processing(list)
def get_markers(screenings):
    coords_to_url = {}
    
    for s in screenings:
        theater = s.get('theater')
        if not theater:
            continue
        lat, lng = theater.get('lat'), theater.get('lon')
        if lat is None or lng is None:
            continue
            
        location = (lat, lng)
        
        url = s['url']
        title = s['map_title']

        message = HTML(f"<a href='{ url }' target='_new'>{ title }</a>")
        
        popup = Popup(
            location=location,
            child=message,
            close_button=False,
            auto_close=False,
            close_on_escape_key=False
        )

        yield popup

m = Map(center=(lat, lng), zoom=12)

for popup in get_markers(theater_screenings):
    m.add_layer(popup)

m.layout.height = '600px'
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …